##This Runs on the older Pyrosetta bindings
---
---

## Running Symmetry without Perl !!!!!! Gasp!!!! Sacrilege!!!!!

In order for Rosetta to model a symmetric protein and make use of the fancy time saving machinery it has, one must use the perl script written by Frank.... Some call it black magic, others call it a black box.

A while back, thanks to Javier's efforts, he encoded some stuff so that you could *auto* detect Cn symmetries as a mover! This is awesome... but sometimes, for some proteins it just doesnt' work. Much of this code is to his credit though. The C++ dies on some proteins though, it has reasonable, hard coded cutoffs that just don't work for all proteins.

This is an implementation a la Pyrosetta that will allow a researcher to simply upload a protein (which may have Cn) symmetry and after some math, will be able to call symmetric movers, score with symmetric scorefunctions and do sweet symmetry stuff.

Note: This currently will not work for Dn symmetries (Until Stephanie figures that out ;) )

Imagine you have a protein (say KIVD- who works on those anyways?) that has C2 symmetry and at the active site, there are catalytic residues from both chains. Suppose we wanted to design on a related sequence that doesn't have a crystal structures, then we would need to build a homology model. Unfortunately, homology modeling already takes a tremendous amount of compute resources that when you double the size of the protein 541\*2, it simply takes forever. 

Fortunately, the homology modeling code (Yifan & Frank) takes a symmetry definition file.... like the ones that come from Franks perl script (make_symm_def_file.pl). However, when you create 1 symmetry definition file, the symmetry is static!  Why is this a problem for us? Well, say I am creating a homology model of a relative of 2vbg, and there are 10 available templates. Each template might have a different way of interacting with at the interface and if I ran each template through Frank's perl script, I would get 10 different answers (slightly different).

So, because in the future I think homology modeling of symmetric interfaces should be able to A) use mutiple templates and B) be able to sample these different orientations and symmetries at the same time, I wrote this notebook. 

In the near future, imagine modeling symmetric interfaces and sample different interface and symmetry perturbations at interfaces... It might help us make better models


## Setup the Functions and Maths needed for symmetry
This essentially is a copy of some of the code in src/numeric/xyzfunctions.hh

There are no bindings for these because they are multi templates, ideally there would be bindings for all core types (and std types)

The following set of functions take an xyzVector and calculates the projections onto different x, y and z axis

Note: In order for Rosetta's symmetry machinery to work, it assumes the center of mass of the Pose is at the origin (0,0,0). See the symmetry documentation for more details, or compare what happens here with what happens when you run the perl script (load the input files and the files that are created to be ran in rosetta)

In [1]:
# This is needed for some builds
from rosetta.protocols.rigid import *
from rosetta.core.pose.symmetry import *
from rosetta.numeric import *
import rosetta.numeric as numeric

## turns out i need some private inline functions
#	inline core::Real angle_with_x_axis_proj_y( xyzVector const & v) const { return numeric::dihedral_degrees(xyzVector(v[0],1,v[2]), xyzVector(0,1,0), xyzVector(0,0,0), xyzVector(1,0,0)); }
def angle_with_x_axis_proj_y( some_xyzVector ):
    return numeric.dihedral_degrees( numeric.xyzVector_double( some_xyzVector[0],1,some_xyzVector[2]), numeric.xyzVector_double(0,1,0), numeric.xyzVector_double( 0,0,0 ), numeric.xyzVector_double( 1,0,0 ) )

def angle_with_y_axis_proj_x( some_xyzVector ):
    return numeric.dihedral_degrees( numeric.xyzVector_double( 1,some_xyzVector[1],some_xyzVector[2]), numeric.xyzVector_double(1,0,0), numeric.xyzVector_double( 0,0,0 ), numeric.xyzVector_double( 0,1,0 ) )

def angle_with_y_axis_proj_z( some_xyzVector ):
    return numeric.dihedral_degrees( numeric.xyzVector_double( some_xyzVector[0],some_xyzVector[1],1), numeric.xyzVector_double(0,0,1), numeric.xyzVector_double( 0,0,0 ), numeric.xyzVector_double( 0,1,0 ) )

def angle_with_z_axis_proj_y( some_xyzVector ):
    return numeric.dihedral_degrees( numeric.xyzVector_double( some_xyzVector[0],1,some_xyzVector[2]), numeric.xyzVector_double(0,1,0), numeric.xyzVector_double( 0,0,0 ), numeric.xyzVector_double( 0,0,1 ) )


This next section copies other code, again mostly for the reason that there are no python bindings that interface with the c++ code correctly.

These create rotation matrices when you input a degree of rotation in degrees

In [2]:
import math
test = numeric.xyzMatrix_double()

def x_rotation_matrix( theta ):
    sin_theta = math.sin(theta)
    cos_theta = math.cos(theta)
    mat = numeric.xyzMatrix_double()
    print "made mat"
    print mat
    
    newmat = mat.rows( 1, 0, 0,
            0, cos_theta, -sin_theta,
             0, sin_theta, cos_theta
            )
    print "changed mat"
    print newmat
    return newmat
    
def x_rotation_matrix_degrees( theta ):
    return x_rotation_matrix( math.radians(theta))

def z_rotation_matrix( theta ):
    sin_theta = math.sin(theta)
    cos_theta = math.cos(theta)
    mat = numeric.xyzMatrix_double()
    print "made mat"
    print mat
    
    newmat = mat.rows( cos_theta, -sin_theta, 0,
            sin_theta, cos_theta, 0,
             0, 0, 1
            )
    print "changed mat"
    print newmat
    return newmat
    
def z_rotation_matrix_degrees( theta ):
    return z_rotation_matrix( math.radians(theta))

def y_rotation_matrix( theta ):
    sin_theta = math.sin(theta)
    cos_theta = math.cos(theta)
    mat = numeric.xyzMatrix_double()
    print "made mat"
    print mat
    
    newmat = mat.rows( cos_theta, 0 , sin_theta,
            0, 1, 0,
             -sin_theta, 0, cos_theta
            )
    print "changed mat"
    print newmat
    return newmat
    
def y_rotation_matrix_degrees( theta ):
    return y_rotation_matrix( math.radians(theta))


## This is an example
f = x_rotation_matrix_degrees( 180 )
print type(f)

made mat
 6.752605561399829E+112  6.946849665632721E-310  1.283039866190166E-316
      0.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000       0.000000000000000

changed mat
      1.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000      -1.000000000000000  -1.224646799147353E-16
      0.000000000000000   1.224646799147353E-16      -1.000000000000000

<class 'rosetta.numeric._numeric_.xyzMatrix_Real'>


---
---


# Let's begin with rosetta

I highly recommend that before you start, you also open Pymol & plot the xyz axes

Once you can see the protein (see next few cells) be sure to clean up (cartoon and color by chain)

In [3]:
from rosetta import *

In [4]:
rosetta.init('-ignore_unrecognized_res T -ignore_waters T -preserve_header T')

PYROSETTA_DATABASE environment variable was set to: /home/steve/Pyrosetta_testing/Rosetta/main/source/build/PyRosetta/linux/namespace/debug/database; using it....
PyRosetta 2014 [Rosetta 2014 unknown:a624fec6942d46d5c86620051caf250f27ea9ba1] retrieved from: 
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Rosetta version  from 
core.init: command: PyRosetta -ignore_unrecognized_res T -ignore_waters T -preserve_header T -database /home/steve/Pyrosetta_testing/Rosetta/main/source/build/PyRosetta/linux/namespace/debug/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=1968208516 seed_offset=0 real_seed=1968208516
core.init.random: RandomGenerator:init: Normal mode, seed=1968208516 RG_type=mt19937


In [5]:
from sjb_util import tmalign

In [6]:
# Read in a pose and setup the pymol observer, you have to score it to get it into pymol

p = pose_from_file('2VBF_cleaned.pdb')
pyobs = PyMOL_Observer() 
pyobs.add_observer(p) 
sfxn = get_fa_scorefxn()
sfxn(p) 


core.chemical.ResidueTypeSet: Finished initializing fa_standard residue type set.  Created 385 residue types
core.chemical.ResidueTypeSet: Total time to initialize 3.57138 seconds.
core.io.pose_from_sfr.PoseFromSFRBuilder: PDB reader is ignoring atom  CD  in residue 172 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: PDB reader is ignoring atom  CE  in residue 172 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: PDB reader is ignoring atom  NZ  in residue 172 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: PDB reader is ignoring atom  CG  in residue 174 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRBuilder: PDB reader is ignoring atom  CD  in residue 174 A.  Pass flag -ignore_zero_occupancy false to change this behavior
core.io.pose_from_sfr.PoseFromSFRB

-706.5423798850721

Notice that the protein is not at the origin

In [7]:
## Take a look at your protein and the wierd chain endings
print p
print p.conformation().chain_endings()

# Fix the weird chain endings (so that later, we can create a nice foldtree)
p.update_pose_chains_from_pdb_chains()
print p.chain_sequence(1)
print p.conformation().chain_endings()

PDB file name: 2VBF_cleaned.pdb
Total residues:1082
Sequence: MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISREDMKWIGNANELNASYMADGYARTKKAAAFLTTFGVGELSAINGLAGSYAENLPVVEIVGSPTSKVQNDGKFVHHTLADGDFKHFMKMHEPVTAARTLLTAENATYEIDRVLSQLLKERKPVYINLPVDVAAAKAEKPALSLENTTEQVILSKIEESLKNAQKPVVIAGHEVISFGLEKTVTQFVSETKLPITTLNFGKSAVDESLPSFLGIYNGKLSEISLKNFVESADFILMLGVKLTDSSTGAFTHHLDENKMISLNIDEGIIFNKVVEDFDFRAVVSSLSELKGIEYEGQYIDKQYEEFIPSSAPLSQDRLWQAVESLTQSNETIVAEQGTSFFGASTIFLKSNSRFIGQPLWGSIGYTFPAALGSQIADKESRHLLFIGDGSLQLTVQELGLSIREKLNPICFIINNDGYTVEREIHGPTQSYNDIPMWNYSKLPETFGATEDRVVSKIVRTENEFVSVMKEAQADVNRMYWIELVLEKEDAPKLLKKMGKLFAEQNKMYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISREDMKWIGNANELNASYMADGYARTKKAAAFLTTFGVGELSAINGLAGSYAENLPVVEIVGSPTSKVQNDGKFVHHTLADGDFKHFMKMHEPVTAARTLLTAENATYEIDRVLSQLLKERKPVYINLPVDVAAAKAEKPALSLENTTEQVILSKIEESLKNAQKPVVIAGHEVISFGLEKTVTQFVSETKLPITTLNFGKSAVDESLPSFLGIYNGKLSEISLKNFVESADFILMLGVKLTDSSTGAFTHHLDENKMISLNIDEGIIFNKVVEDFDFRAVVSSLSELKGIEYEGQYIDKQYEEFIPSSAPLSQDRLWQAVESLTQSNETIVAEQGTSFFGASTIFLKSNSRFIGQPLWGS

In [8]:
#### Now starts the pyrosetta implementation of DetectSymmetry:apply
## We might be able to skip this if we can pull the data from the mmcif pdb file or if
## we know a priori what kind of symmetry exists in the family of proteins we want to model
n_jumps = p.num_jump()
symmetric_type = n_jumps +1
seq1 = p.chain_sequence(1)
#pose made from chain A seq
print len(seq1)
new_pose = Pose(p, 1, len(seq1))
print new_pose

541
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 181 in file . Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 182 in file . Best match rsd_type:  ASN
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 181 atom  OXT (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 1H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 2H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 3H   (trying to store temperature in PDBInfo)
PDB file name: 
Total residues:541
Sequence: MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISREDMKWIGNANELNASYMADGYARTKKAAAFLTTFGVGELSAINGLAGSYAENLPVVEIVGSPTSKVQNDGKFVHH

In [9]:
## This goes through each of the subunits (different chains) and calculates the rms of the chain A
## to chain X, this is to make sure all of the 'symmetric' chains are actually symmetric

i = 1
while i != symmetric_type:
    #i +=1
    if len(seq1) != len(p.chain_sequence(2)):
        print "Subunits have different length sequences"
    
    test_pose = Pose(p, i*len(seq1)+1,(i+1)*len(seq1))
    rms = core.scoring.CA_rmsd(new_pose,test_pose)
    print rms
    i +=1

core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 181 in file . Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 182 in file . Best match rsd_type:  ASN
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 181 atom  OXT (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 1H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 2H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 3H   (trying to store temperature in PDBInfo)
0.374408006668
nan
nan


The next few cells first move the input protein to the origin (COM). But then the next few apply rotations that align the COM of each chain with the XY, XZ and YZ planes

In [10]:
### This first move moves the proteins COM to the origin (0,0,0)
id_rot_mat = numeric.xyzMatrix_Real.identity()
cm_pose = core.pose.center_of_mass(p,1,p.total_residue())
print "starting cm"
print cm_pose

## So this is just applying the negative of the com to the pose (so a origin placement)
p.apply_transform_Rx_plus_v(id_rot_mat, -1*cm_pose)
print id_rot_mat
#sfxn(p)
print "newcom"
newcom = core.pose.center_of_mass(p,1,p.total_residue())
print newcom

starting cm
      22.55059149722738       100.9876136783734       17.64648706099815
      1.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       1.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000       1.000000000000000

newcom
 -2.047899742576592E-14   7.538845292537846E-15   9.916076996282359E-16


In [11]:
## align the com of chain A with Y axis
#first, rotate around x to align com of chain A with xy plane
cm_chain_A = core.pose.center_of_mass(p,1,len(seq1))
angle_y1 = angle_with_y_axis_proj_x( cm_chain_A )
x_rot = x_rotation_matrix_degrees( -1*angle_y1 )
p.apply_transform_Rx_plus_v( x_rot, numeric.xyzVector_double(0,0,0))
new_cm_chain_A = core.pose.center_of_mass(p,1,p.total_residue())
print new_cm_chain_A
print "plane tolerance: "
print new_cm_chain_A[2]
if (new_cm_chain_A[2] > -1*0.05) and (new_cm_chain_A[2] < .05):
    print "pass plane check x"
#sfxn(p)

made mat
      0.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000  4.743030200075967E-322

changed mat
      1.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000     -0.9907564650779119     -0.1356525963862112
      0.000000000000000      0.1356525963862112     -0.9907564650779119

 -2.047899742576592E-14  -4.715061777040221E-15  -1.076977898933978E-15
plane tolerance: 
-1.07697789893e-15
pass plane check x


In [12]:
 ## now rotat along Z axis   
angle_y2 = angle_with_y_axis_proj_z(  new_cm_chain_A )
z_rot = z_rotation_matrix_degrees( -1*angle_y2)
p.apply_transform_Rx_plus_v( z_rot, numeric.xyzVector_double(0,0,0))
new2_cm_chain_A = core.pose.center_of_mass(p,1,p.total_residue())
print new2_cm_chain_A[2]
if (new2_cm_chain_A[2] > -1*0.05) and (new2_cm_chain_A[2] < .05):
    print "pass plane check z"


made mat
      0.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000  4.743030200075967E-322

changed mat
    -0.2243687965295555      0.9745043063752356       0.000000000000000
    -0.9745043063752356     -0.2243687965295555       0.000000000000000
      0.000000000000000       0.000000000000000       1.000000000000000

-1.07697789893e-15
pass plane check z


Note now that the protein is centered over the origin, and has undergone some rotations

This next cell is just a sanity check that the com of the chain B is also near 0

In [13]:
## now check the other chain's com
i = 1
cm_chain = rosetta.core.pose.center_of_mass(p, i*len(seq1)+1,(i+1)*len(seq1))
print "Chain B"
print cm_chain[2]

Chain B
-4.72819565386e-16


In [14]:
## Now perform C2 check plus one last rotation
vm = ( p.residue(1).xyz("CA") + p.residue(len(seq1)+1).xyz("CA"))/2
print vm 
angle_z_axis = angle_with_z_axis_proj_y( vm )
rot1 = y_rotation_matrix_degrees( -1*angle_z_axis )
print angle_z_axis 
print rot1 
print '-'*30
p.apply_transform_Rx_plus_v( rot1, numeric.xyzVector_double(0,0,0))
vm_new = ( p.residue(1).xyz("CA") + p.residue(len(seq1)+1).xyz("CA"))/2
print "This should be less than .001"
print vm_new[0]
print "This too "
print vm_new[1]

print '-'*30
print vm_new

      1.740234172849955      -7.256881989694905       2.412365276238608
made mat
      0.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000       0.000000000000000
      0.000000000000000       0.000000000000000  4.743030200075967E-322

changed mat
     0.8110029714872337       0.000000000000000     -0.5850420328821487
      0.000000000000000       1.000000000000000       0.000000000000000
     0.5850420328821487       0.000000000000000      0.8110029714872337

35.8059594973
     0.8110029714872337       0.000000000000000     -0.5850420328821487
      0.000000000000000       1.000000000000000       0.000000000000000
     0.5850420328821487       0.000000000000000      0.8110029714872337

------------------------------
This should be less than .001
0.0
This too 
-7.25688198969
------------------------------
      0.000000000000000      -7.256881989694905       2.974545545517254


In [15]:
# Now, since the protein is in the right place, we just make a copy of the chain A
alignpose = Pose(p,1,len(seq1))

core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 181 in file . Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 182 in file . Best match rsd_type:  ASN
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 181 atom  OXT (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 1H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 2H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 182 atom 3H   (trying to store temperature in PDBInfo)


In [16]:
# here, we read in the generic sym files in the Rosetta database
# We can use ChainA and the symdef information to create a symmetric pose!
from rosetta.basic import *
from rosetta.core.conformation.symmetry import *

c2_symm = 'C2_Z.sym'
# the sym file is usually located at the below path
#c2_symm = '~/Rosetta/main/database/symmetry/cyclic/C2_Z.sym'
symdef = core.conformation.symmetry.SymmData()
symdef.read_symmetry_data_from_file( c2_symm )

#p = 
core.pose.symmetry.make_symmetric_pose( alignpose, symdef )
p = alignpose

core.conformation.symmetry.SymmData: Setting weight of master jump ( jump-id=4 ) to 1.0 (was undefined)
core.conformation.symmetry.SymmData: Setting jump_group JGS: [master 4]  5:0 
core.conformation.symmetry.SymmData: Setting weight of master jump ( jump-id=1 ) to 1.0 (was undefined)
core.conformation.symmetry.SymmData: Setting jump_group JGSS: [master 1]  2:0 
core.conformation.symmetry.SymmData: symmetry name: C2
core.conformation.symmetry.SymmData: number of subunits: 2
core.conformation.symmetry.SymmData: number of interfaces: 1
core.conformation.symmetry.SymmData: score subunit number: C1
core.conformation.symmetry.SymmData: anchor the subunits at residue: COM
core.conformation.symmetry.SymmData:  Virtual coordinate system C1
core.conformation.symmetry.SymmData: x: 0 0 1
core.conformation.symmetry.SymmData: y: 0 1 0
core.conformation.symmetry.SymmData: origin: 0 0 0
core.conformation.symmetry.SymmData:  Virtual coordinate system C2
core.conformation.symmetry.SymmData: x: 0 0 1
co

##
Wells, that's it!

You should have seen a bunch of symmetry stuff flash on the screen
We now have a pose inside of rosetta that is symmetric!!! And not an ounce of Perl was used!

So we can do all the normal stuff like symmetric packing
symmetric scoring, etc

In [17]:
import rosetta.core.conformation.symmetry
import rosetta.core.pose.symmetry
from rosetta.core.scoring.symmetry import *
import rosetta.protocols.simple_moves.symmetry


#print p
symopts = rosetta.core.conformation.symmetry.SymmDataOptions()
#rosetta.core.conformation.symmetry.SymmetryInfo()
#So rosetta sees this as a symmetric pose
rosetta.core.conformation.symmetry.is_symmetric(p.conformation())


sym_sfxn = rosetta.core.scoring.symmetry.SymmetricScoreFunction()
sym_packer = protocols.simple_moves.symmetry.SymPackRotamersMover( sym_sfxn )
sym_min = protocols.simple_moves.symmetry.SymMinMover()
mm = MoveMap()
core.pose.symmetry.make_symmetric_movemap( p, mm)
#sym_packer.apply(p)

# But now how do I score it?
# and do i need to turn other options on?
#symopts.show()

In [18]:
print rosetta.core.conformation.symmetry.is_symmetric(p.conformation())

True


this pyrosetta lesson was brought to you by Steve Bertolani